In [1]:
import pandas as pd
import lightfm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k, reciprocal_rank, auc_score
from lightfm.cross_validation import random_train_test_split
from dataloading import DataLoader
from datapreparation_py import DataPreparation
from lightFM_adoptor import LightFMAdaptor

c:\Users\781110104\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

C:\Users\781110104\OneDrive - Genpact\Documents\VSCode


In [3]:
dl = DataLoader()
user_item_co_t = dl.import_data('TEREX', 'CO', 'df')

In [4]:
user_item_co_t_filtered = dl.remove_low_interact_items(user_item_co_t, 1)

In [5]:
dp = DataPreparation(user_item_co_t_filtered)

In [6]:
user_features = ['country', 'brand']
item_features = ['group2', 'movement_code']
input_data = dp.get_input_data(user_features=user_features, item_features=item_features)

In [7]:
interactions, weights, userf, itemf = input_data

In [8]:
train_i, test_i = random_train_test_split(interactions, random_state=22)
train_w, test_w = random_train_test_split(weights, random_state=22)

In [9]:
model_adopt = LightFMAdaptor(epochs=15, loss='warp', random_state=22, no_components=64)

In [10]:
model_adopt.fit(item_users=train_i, user_features=userf, item_features=itemf, sample_weight=train_w)

Epoch: 100%|██████████| 15/15 [00:16<00:00,  1.12s/it]


In [11]:
model = LightFM(loss='warp', no_components=64, random_state=22)
model.fit(interactions=train_i, user_features=userf, item_features=itemf, sample_weight=train_w, epochs=15)

In [15]:
model.user_embeddings.shape

(79, 64)

In [20]:
num_users = userf.shape[0]
num_features = userf.shape[1] - userf.shape[0]
num_features

39

In [21]:
num_users = userf.shape[0]
num_features = userf.shape[1] - userf.shape[0]
return_embeddings = model.user_embeddings[:num_users, :]
for u in range(num_users):
    for i in range(num_features):
        return_embeddings[u, :] += userf.tocsr()[u, num_features+i] * return_embeddings[u, :]

In [49]:
itemf.shape

(16690, 16715)

In [50]:
num_items = itemf.shape[0]
num_features = itemf.shape[1] - itemf.shape[0]
return_embeddings_i = model.item_embeddings[:num_items, :]
for u in range(num_items):
    for i in range(num_features):
        return_embeddings_i[u, :] += itemf.tocsr()[u, num_features+i] * return_embeddings_i[u, :]


In [53]:
model_adopt.user_factors = return_embeddings
model_adopt.item_factors = return_embeddings_i

In [51]:
return_embeddings_i.shape

(16690, 64)

In [52]:
model_adopt.item_factors.shape

(16690, 66)

In [38]:
userf.tocsr()[0, num_features+1] * return_embeddings[0, :]

1.0

In [47]:
return_embeddings.shape

(40, 64)

In [ ]:
return_embeddings_fac = np.concatenate((return_embeddings, model.user_biases[:users].reshape(users, 1), np.ones((users, 1))), axis=1).copy()

In [46]:
model.predict_rank()

(40, 66)

In [ ]:
def normalize(csr):
    lil = csr.tolil()
    for i in range(csr.shape[0]):
        array = csr[i, csr.shape[0]:].toarray()
        norm_array = array / array.sum()
        lil[i, csr.shape[0]:] = norm_array
    return lil.tocsr()

In [28]:
userf.tocsr().shape

(40, 79)

In [25]:
userf.tocsr()[0, num_features+0]

0.0

In [48]:
test_auc = auc_score(model, test_interactions=test_i, train_interactions=train_i).mean()
test_auc

0.8196564

In [13]:
test_pk = precision_at_k(model, test_interactions=test_i, user_features=userf, item_features=itemf, train_interactions=train_i).mean()
test_pk

0.225

In [54]:
from implicit.evaluation import ranking_metrics_at_k
ranking_metrics_at_k(model_adopt, train_i, test_i, 10)

  0%|          | 0/40 [00:00<?, ?it/s]

{'precision': 0.0425,
 'map': 0.020070436507936506,
 'ndcg': 0.043128915134309685,
 'auc': 0.5015753453258147}

In [34]:
test_rk = recall_at_k(model, test_interactions=test_i, train_interactions=train_i, user_features=userf, item_features=itemf).mean()
test_rk

0.010872608434937672

In [40]:
test_rr = reciprocal_rank(model, test_interactions=test_i, train_interactions=train_i, user_features=userf, item_features=itemf).mean()
test_rr

0.46444315